In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

In [4]:
df_cf = pd.read_parquet(r'C:\Projects\Recomender System\data\processed\ratings_for_cf.parquet')
df_cf.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [5]:
user_ids = df_cf['userId'].unique()
movie_ids = df_cf['movieId'].unique()

user_id_to_idx = {u: i for i, u in enumerate(user_ids)}
movie_id_to_idx = {m: i for i, m in enumerate(movie_ids)}

n_users = len(user_ids)
n_movies = len(movie_ids)

rows = df_cf['userId'].map(user_id_to_idx)
cols = df_cf['movieId'].map(movie_id_to_idx)
data = df_cf['rating'].astype(float)

R = csr_matrix((data, (rows, cols)), shape=(n_users, n_movies))
R

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24945870 stored elements and shape (162541, 32720)>

In [ ]:
k = 50
nmf = NMF(
    n_components=k,
    init='random',
    random_state=42,
    max_iter=200
)

U = nmf.fit_transform(R)
V = nmf.components_

c:\Users\Innolab\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\decomposition\_nmf.py:1728: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


KeyboardInterrupt: 